# Module 4 - Query MongoDB (35 minutes)

It may not seem as easy to query as with SQL, but once you get used to the process querying data with a NoSQL database like MongoDB is pretty straightforward and very flexible.

* Please feel free to ask questions at any time!

In [ ]:
!pip install pymongo

In [ ]:
from pymongo import MongoClient

In [ ]:
# assuming 'cars' collection exists

client = MongoClient('localhost', port=27017)  # shake hands with your PC
db = client.test  # connect to the 'test' instance
cars = db.cars  # connect to the 'cars' collection

A great feature (in my opinion) of MongoDB is that it automatically creates a new instance even if one doesn't exist. Also, it automatically creates a new collection even if one doesn't exist. I have never worked with a relational database that works in this manner. Actually, creating new instances in relational databases (like Oracle) is a real pain in the keester. 

# Create a NoSQL query with MongoDB

In [ ]:
# create line feed variable for convenience

br = '\n'

# establish number of documents to display

n = 1

# create query

q = cars.find().limit(n)

# display documents to limit of n

[row for row in q]  # use list comprehension to display results of the query

## We just created a simple NoSQL query!
### A NoSQL query with MongoDB has a few moving parts:

1. start with the collection name 'cars'
2. use the 'find()' method to return all records from 'cars'
3. add the 'limit()' method to limit output to 1 record
4. finally, display the results of query 'q'

### Notice that a NoSQL query has a pretty logical and sequential layout. It begins with the collection name followed by the 'find()' method. The 'find()' method is key because it controls what records are returned. The 'find()' method can be augmented with other submethods like 'limit()'.

# Let's create another query

In [ ]:
n = 3

# create query

q = cars.find().limit(n)

# anatomy of result set

# display first document

print (q[0], br)

# display third document

print (q[2], br)

# display pieces of third document by keys

print (q[2]['Car'], end = ' ')
print ('Model ' + str(q[2]['Model']), end = ' ')
print ('Made in the ' + '\'' + str(q[2]['Origin']) + 'A' + '\'')

## We just queried all cars, but limited output to 3 records

Although we return all records from the collection, We limit output to just 3 records for convenience. We can access each record individually by an index value. So, the first record is accessed by index 0, second by index 1, and third by index 2. We can also slice off pieces (fields) within each record by using their keys. So, we project 'Car', 'Model', and 'Origin' keys and piece the result with some text to make it look nicer.

* We want to limit the records returned because the dataset includes hundreds of cars! 

# Let's move onto 'inclusion'

We can **include** keys in our query with the key name, a ':', and a '1'. The '1' tells Python that we want to include this key.

In [ ]:
# inclusion

n = 5

print ('1st', n, 'cars:')

p = {'Car':1, 'MPG':1, 'HP':1}

q = cars.find({}, p).limit(n)
[row for row in q]

## The '{}' in the 'find()' method tells Python to return all records from the collection.

We build add the fields we want in our query and assign to variable 'p'. We then include variable 'p' in the 'find()' method. Finally, we display results with list comprehension.

# Now, let's demonstrate 'exclusion'

In [ ]:
# exclusion

n = 5

print ('1st', n, 'cars:')

p = {'_id':0, 'Model':0, 'Acceleration':0, 'Weight':0,
     'Displacement':0, 'MPG':0, 'Origin':0}

q = cars.find({}, p).limit(n)
[row for row in q]

## Use '0' instead of '1' to exclude fields.

The logic is the same as with inclusion, but we explicitly exclude the fields we don't want with '0'. So, all fields except the ones we excluded are projected in the output.

# We can easily skip documents

In [ ]:
# skip documents

n = 3
hop = 200

print ('Skip', hop, 'cars:')

p = {'Model':0, 'Acceleration':0, 'Weight':0,
     'Displacement':0, 'MPG':0, 'Origin':0}

q = cars.find({}, p).skip(hop).limit(n)

[row for row in q]

## Use the 'skip' method to skip over any number of documents

# Query with selector operators

The '$in' operator is used to include records with matching values. So, we can query all Japanese cars with the 'q_japan' query.

In [ ]:
# query with selector operators

n = 5

# return all Japanese cars

q_japan = cars.find({'Origin' :
                     {'$in' : ['Japan']}}).limit(n)

# display n documents

[(row['_id'], row['Car'], row['Origin']) for row in q_japan]

## Notice that we place our filters inside the 'find()' method to add more complexity to the query.

# Let's create a more complex query

In [ ]:
# return Europe and Japan cars with HP > 113,
# Weight > 2000, with 4 or 6 cylinders

q_hp_acc_origin = cars.find({
    '$and':[
        {'HP': {'$gt':113}},
        {'Weight': {'$gt':2000}},
        {'$or':[
            {'Cylinders':4},
            {'Cylinders':6}
            ]
         },
        {'Origin':{'$in':['Europe', 'Japan']}}
        ]
    })

# display all of them

[(row['Car'], row['Origin'], row['HP'],
  row['Cylinders'], row['Weight'])
 for row in q_hp_acc_origin]

We do a whole bunch of things with this query. We want to retrieve all records with 'HP' greater than '113' AND 'Weight' greater than '2000'. But, only include those records returned that have either 4 or 6 cylinder engines or originate from Europe or Japan. Although the query looks complex, it is not really that bad once you get used to the structure.

Notice that the outside '\\$and' is key to the whole query. The main query has 4 logical filter pieces all of which have to be satisfied since we are using the logical 'AND'. The four filter pieces include 'HP', 'Weight', 'Cylinders', <strong><font color=red>and</font></strong> 'Origin'. The 'Cylinders' filter uses an '\\$or' to limit to either '4' or '6'. The 'Origins' filter uses an '\\$in' to limit to either 'Europe' or 'Japan'. We could also use '\\$nin' with 'US' to get the same result. 

We can simplify the query because 'AND' logic is implied! We can also replace the 'OR' logic with '\\$in'.

In [ ]:
q_hp_acc_origin = cars.find({
    'HP': {'$gt':113},
    'Weight': {'$gt':2000},
    'Cylinders':{'$in':[4, 6]},
    'Origin':{'$in':['Europe', 'Japan']}
    })

# display all of them

[(row['Car'], row['Origin'], row['HP'],
  row['Cylinders'], row['Weight'])
 for row in q_hp_acc_origin]

* There is nothing wrong with the original query, but don't you think that this one is much easier?

# The next few queries are much easier

### Return 'Japan' cars with 'MPG' between '37' and '39':

In [ ]:
# return Japanese cars with MPG between 37 and 39

q_between_mpg = cars.find({
    'Origin': 'Japan', 'MPG': {'$lt':39, '$gt':37}
    })

[(row['Car'], row['MPG'], row['Origin'])
 for row in q_between_mpg]

### Return all cars with 'MPG' greater than or equal to '40':

In [ ]:
# return all cars with MPG > 40

q_hi_mpg = cars.find({'MPG': {'$gte':40}})

[(row['Car'], row['MPG'], row['Origin'])
 for row in q_hi_mpg]

### Return all cars with either '3' or '5' cylinders:

In [ ]:
# query cars with odd number of cylinder

q_odd_cyl = cars.find({'Cylinders':{'$in':[3, 5]}})

[(row['Car'], row['Origin'], row['HP'],
  row['Cylinders'], row['Weight'])
 for row in q_odd_cyl]

# Here's some heuristics to follow when creating complex queries

Of course, queries can be as complex as you wish to make them. When creating a very complex query, we suggest:

1. start with a clear plan of what you wish to query
2. break your planned query into smaller ones
3. get the smaller queries to work
4. start piecing together the smaller queries into larger ones.
5. debug as you add more logic to the query

When we get frustrated building complex queries, we follow the steps just presented. Even the complex query in this lesson took us some work!

# Module 4 Exercise

Create a query with the following specifications:

* Exclude cars made in the United States
* Weight must be between 1750 and 2200 pounds (inclusive)
* Acceleration must be greater than 15
* MPG must be greater than 25

# Our solution

* Begin with a plan. We have four conditions that we must meet. Let's begin with the first condition:

In [ ]:
q = cars.find({
    'Origin': {'$nin': ['US']}
    })

### We mentioned the 'nin' operator, but didn't use it in an example. 

In [ ]:
# verify results

[row['Origin'] for i, row in enumerate(q) if i < 3]

* Continue by adding the second condition:

In [ ]:
q = cars.find({
    'Origin': {'$nin':['US']},
    'Weight': {'$gte':1750, '$lte':2200}
    })

In [ ]:
# verify results

[(row['Origin'], row['Weight']) for i, row in enumerate(q) if i < 3]

* Let's go ahead and add the third and fourth condition

In [ ]:
q = cars.find({
    'Origin': {'$nin':['US']},
    'Weight': {'$gte':1750, '$lte':2200},
    'Acceleration': {'$gt':15},
    'MPG': {'$gt':25}
    })

In [ ]:
# verify results

[(row['Origin'], row['Weight'], row['Acceleration'], row['MPG'])
 for i, row in enumerate(q) if i < 3]

# What did we learn?

1. we read a CSV file into a Pandas DataFrame
2. we converted the DataFrame into dictionary elements
3. we dumped the data to JSON
4. we read the JSON data to verify that all worked as planned
5. we sharpened our querying skills with an exercise

## Questions?

# <font color=red>5 minute break</font>